In [4]:
import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations

psi4.core.set_output_file("h2o2.out")

h2o2 = psi4.geometry("""0 1
O        0.000000    0.695000   -0.092486
O       -0.000000   -0.695000   -0.092486
H       -0.388142    0.895249    0.739888
H        0.388142   -0.895249    0.739888
symmetry c1
""", name="H2O2")

psi4.set_options({
    'save_jk': True,
})

e, wfn = psi4.energy("HF/cc-pvdz", return_wfn=True, molecule=h2o2)
res = tdscf_excitations(wfn, states=10)

In [5]:
import numpy as np

import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from psi4.driver.p4util import spectrum

psi4.core.set_output_file("moxy.out")

moxy = psi4.geometry("""0 1
C  0.152133 -0.035800  0.485797
C -1.039475  0.615938 -0.061249
C  1.507144  0.097806 -0.148460
O -0.828215 -0.788248 -0.239431
H  0.153725 -0.249258  1.552136
H -1.863178  0.881921  0.593333
H -0.949807  1.214210 -0.962771
H  2.076806 -0.826189 -0.036671
H  2.074465  0.901788  0.325106
H  1.414895  0.315852 -1.212218
""", name="(S)-methyloxirane")

psi4.set_options({
    'save_jk': True,
})

e, wfn = psi4.energy("HF/cc-pvdz", return_wfn=True, molecule=moxy)
res = tdscf_excitations(wfn, states=8, triplets="also")

# get poles and residues to plot OPA and ECD spectra
poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["LENGTH-GAUGE ELECTRIC DIPOLE TRANSITION MOMENT"])**2 for r in res]
ecd_residues = [r["LENGTH-GAUGE ROTATORY STRENGTH"] for r in res]

opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")
ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")

ImportError: cannot import name 'spectrum' from 'psi4.driver.p4util' (c:\programdata\pymol\lib\site-packages\psi4\driver\p4util\__init__.py)

In [11]:
#! td-camb3lyp with DiskDF and method/basis specification

from psi4.driver.procrouting.response.scf_response import tdscf_excitations

psi4.core.set_output_file("h2o.out")

h2o=psi4.geometry("""
O
H 1 r
H 1 r 2 a

r=0.958
a=104.5

symmetry c1
""")


#psi4.set_options({
#    'memory': '280 mb',
#    'scf_type': 'disk_df',
#    'save_jk': True,
#    'df_ints_io': 'save',
#    'e_convergence': 8,
#    'd_convergence': 8
#})

e, wfn = psi4.energy("b3lyp/aug-cc-pvtz", return_wfn=True, molecule=h2o)
res = tdscf_excitations(wfn, states=10)


In [13]:
!more h2o.out

In [ ]:
#! td-uhf test on triplet states of methylene (rpa)

UHF_RPA_cc_pvdz = [0.2445704160468683,
                   0.2878574429978692,
                   0.3179110389232691,
                   0.3547301851175197,
                   0.3879221731828428,
                   0.4038089052916107,
                   0.4352993997260387,
                   0.4508039388809985,
                   0.4834961361605727]

molecule ch2 {
0 3

C           0.000000    0.000000    0.159693
H          -0.000000    0.895527   -0.479080
H          -0.000000   -0.895527   -0.479080
symmetry c1
}

set {
    reference uhf
    scf_type pk
    e_convergence 8
    d_convergence 8
}

set scf tdscf_states [9]

energy('td-hf/cc-pvdz')

for n, ref in enumerate(UHF_RPA_cc_pvdz):
    ex_en = psi4.variable(f"TD-HF ROOT 0 -> ROOT {n+1} EXCITATION ENERGY - A SYMMETRY")
    compare_values(ref,ex_en, 4, f"TD-UHF/cc-pvdz ROOT 0 -> ROOT {n+1} EXCITATION ENERGY - A SYMMETRY")